# Classifiers
Next up, we want to use classifying algorithms to predict the outcome of future matches. To do so, we will need to examine what data we have to make use of the features created in the previous notebook. First we will try to predict the outcome of the matches, we will here use grid search to find the best classifier with the best parameteres. After that, we will also try and predict the outcome in terms of goal difference, as described in the first notebook.